# Imports

In [1]:
from dqn_agent import DQNAgent
from ppo_agent import PPOAgent
from data_processing import preprocess_all_data, load_preprocessed_dataset, get_activity, get_column_units
from visualize import visualize_results, visualize_training, visualize_preprocessed_data
import numpy as np

In [2]:
# Load the preprocessed dataset

df = load_preprocessed_dataset()
print(df.head())

--- Preprocessed Dataset Info ---
Number of rows: 175498
Number of columns: 42
---------------------------------
   Timestamp  Activity ID  Heart Rate  Hand Sensor - Temperature  \
0      37.70            1       100.0                     30.375   
1      37.81            1       100.0                     30.375   
2      37.92            1       100.0                     30.375   
3      38.03            1       100.0                     30.375   
4      38.14            1       101.0                     30.375   

   Hand Sensor - Accelerometer - X  Hand Sensor - Accelerometer - Y  \
0                          2.30106                          7.25857   
1                          2.24615                          7.48180   
2                          2.30000                          7.10681   
3                          2.49455                          7.52335   
4                          2.71654                          8.30596   

   Hand Sensor - Accelerometer - Z  Hand Sensor - G

# Initialize system

In [5]:
dqn_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': 2,  
    'hidden_size': 64,
    'lr': 1e-3,
    'gamma': 0.99,
    'batch_size': 32,
    'memory_size': 10000,
    'epsilon_start': 1.0,
    'epsilon_min': 0.01,
    'epsilon_decay': 0.995
}

ppo_config = {
    'state_size': df.shape[1] - 1,  
    'action_size': 2,  
    'hidden_size': 64,
    'lr': 3e-4,
    'gamma': 0.99,
    'clip_epsilon': 0.2,
    'update_epochs': 4,
    'batch_size': 64,
    'c1': 0.5,
    'c2': 0.01
}

dqn_agent = DQNAgent(**dqn_config)
ppo_agent = PPOAgent(**ppo_config)

# Training

In [6]:
def train_agent(agent, df, agent_name):
    if isinstance(agent, DQNAgent):
        for _ in range(5):
            for index, row in df.iterrows():
                state = row[:-1].values
                action = agent.predict(state)
                reward = row[-1]  # Assuming reward is last column
                next_state = state  # DQNAgent does not generate next state; add logic if needed
                done = False
                agent.remember(state, action, reward, next_state, done)
                agent.train()
    else:  # PPO
        for episode in range(5):
            total_reward = 0
            for index, row in df.iterrows():
                state = row[:-1].values
                action = agent.predict(state)
                reward = row[-1]  # Assuming reward is last column
                done = False
                agent.store_transition(state, action, agent.last_log_prob, reward, done, agent.last_value)
            agent.train()

print("Training DQN Agent...")
dqn_agent.save("dqn_trained_model.pth")

print("Training PPO Agent...")
ppo_agent.save("ppo_trained_model.pth")

Training DQN Agent...
Training PPO Agent...


# Testing


In [7]:
def test_agent(agent, df, agent_name):
    total_reward = 0
    for index, row in df.iterrows():
        state = row[:-1].values
        total_reward += agent.predict(state)
    print(f"{agent_name} Test Reward: {total_reward}")

print("Testing DQN Agent...")
dqn_agent.save("dqn_trained_model.pth")
test_agent(dqn_agent, df, "DQN")

print("Testing PPO Agent...")
ppo_agent.save("ppo_trained_model.pth")
test_agent(ppo_agent, df, "PPO")

Testing DQN Agent...


KeyboardInterrupt: 

# Visualize results

In [ ]:
# visualize_training([], [])
# visualize_results(dqn_agent, ppo_agent, df)
# visualize_preprocessed_data(df)